In [58]:
import requests

In [59]:
from bs4 import BeautifulSoup

# 7-days forecast in San Francisco from forecast.weather.gov

In [60]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")

In [61]:
soup = BeautifulSoup(page.content,'html.parser')

In [62]:
seven_day = soup.find(id="seven-day-forecast")

In [63]:
forecast_items = seven_day.find_all(class_="tombstone-container")

## Extracting today forecast

In [64]:
today = forecast_items[0]
print(today.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br>
   <br/>
  </br>
 </p>
 <p>
  <img alt="Today: Cloudy, then gradually becoming mostly sunny, with a high near 62. West wind 7 to 12 mph increasing to 13 to 18 mph in the afternoon. Winds could gust as high as 24 mph. " class="forecast-icon" src="newimages/medium/bkn.png" title="Today: Cloudy, then gradually becoming mostly sunny, with a high near 62. West wind 7 to 12 mph increasing to 13 to 18 mph in the afternoon. Winds could gust as high as 24 mph. "/>
 </p>
 <p class="short-desc">
  Decreasing
  <br>
   Clouds
  </br>
 </p>
 <p class="temp temp-high">
  High: 62 °F
 </p>
</div>


In [65]:
period = today.find(class_="period-name").get_text()
short_desc = today.find(class_="short-desc").get_text()
temp = today.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Today
DecreasingClouds
High: 62 °F


In [66]:
img = today.find("img")

desc = img["title"]
print(desc)

Today: Cloudy, then gradually becoming mostly sunny, with a high near 62. West wind 7 to 12 mph increasing to 13 to 18 mph in the afternoon. Winds could gust as high as 24 mph. 


## Extracting 7-days forecast

In [67]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday']

In [68]:
short_desc = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
desc = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_desc)
print(temps)
print(desc)

['DecreasingClouds', 'IncreasingClouds', 'DecreasingClouds', 'Mostly Cloudy', 'Partly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny']
['High: 62 °F', 'Low: 50 °F', 'High: 61 °F', 'Low: 51 °F', 'High: 60 °F', 'Low: 51 °F', 'High: 61 °F', 'Low: 51 °F', 'High: 61 °F']
['Today: Cloudy, then gradually becoming mostly sunny, with a high near 62. West wind 7 to 12 mph increasing to 13 to 18 mph in the afternoon. Winds could gust as high as 24 mph. ', 'Tonight: Increasing clouds, with a low around 50. West southwest wind 9 to 18 mph, with gusts as high as 24 mph. ', 'Thursday: Cloudy, then gradually becoming mostly sunny, with a high near 61. West wind 9 to 14 mph increasing to 16 to 21 mph in the afternoon. Winds could gust as high as 26 mph. ', 'Thursday Night: Mostly cloudy, with a low around 51. West wind 16 to 20 mph, with gusts as high as 25 mph. ', 'Friday: Partly sunny, with a high near 60. West wind 10 to 15 mph, with gusts as high as 20 mph. ', 'Friday Night

 ## Combining data into a Pandas Dataframe

In [69]:
import pandas as pd

In [70]:
weather = pd.DataFrame({"period":periods,
                       "short_desc":short_desc,
                       "temp":temps,
                       "desc":desc})
weather

,desc,period,short_desc,temp
0,"Today: Cloudy, then gradually becoming mostly ...",Today,DecreasingClouds,High: 62 °F
1,"Tonight: Increasing clouds, with a low around ...",Tonight,IncreasingClouds,Low: 50 °F
2,"Thursday: Cloudy, then gradually becoming most...",Thursday,DecreasingClouds,High: 61 °F
3,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Mostly Cloudy,Low: 51 °F
4,"Friday: Partly sunny, with a high near 60. Wes...",Friday,Partly Sunny,High: 60 °F
5,"Friday Night: Partly cloudy, with a low around...",FridayNight,Partly Cloudy,Low: 51 °F
6,"Saturday: Mostly sunny, with a high near 61.",Saturday,Mostly Sunny,High: 61 °F
7,"Saturday Night: Partly cloudy, with a low arou...",SaturdayNight,Partly Cloudy,Low: 51 °F
8,"Sunday: Mostly sunny, with a high near 61.",Sunday,Mostly Sunny,High: 61 °F


## Cleaning data

In [71]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)",expand=False)
weather["temp_num"] = temp_nums.astype("int")
temp_nums

0    62
1    50
2    61
3    51
4    60
5    51
6    61
7    51
8    61
Name: temp_num, dtype: object

In [72]:
weather = weather.drop(["temp"],axis=1)

In [73]:
weather

,desc,period,short_desc,temp_num
0,"Today: Cloudy, then gradually becoming mostly ...",Today,DecreasingClouds,62
1,"Tonight: Increasing clouds, with a low around ...",Tonight,IncreasingClouds,50
2,"Thursday: Cloudy, then gradually becoming most...",Thursday,DecreasingClouds,61
3,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Mostly Cloudy,51
4,"Friday: Partly sunny, with a high near 60. Wes...",Friday,Partly Sunny,60
5,"Friday Night: Partly cloudy, with a low around...",FridayNight,Partly Cloudy,51
6,"Saturday: Mostly sunny, with a high near 61.",Saturday,Mostly Sunny,61
7,"Saturday Night: Partly cloudy, with a low arou...",SaturdayNight,Partly Cloudy,51
8,"Sunday: Mostly sunny, with a high near 61.",Sunday,Mostly Sunny,61


In [74]:
weather["desc"] = weather["desc"].str.replace("([\w]*[\s]*[\w]*:)([\w]*)","")

In [75]:
weather

,desc,period,short_desc,temp_num
0,"Cloudy, then gradually becoming mostly sunny,...",Today,DecreasingClouds,62
1,"Increasing clouds, with a low around 50. West...",Tonight,IncreasingClouds,50
2,"Cloudy, then gradually becoming mostly sunny,...",Thursday,DecreasingClouds,61
3,"Mostly cloudy, with a low around 51. West win...",ThursdayNight,Mostly Cloudy,51
4,"Partly sunny, with a high near 60. West wind ...",Friday,Partly Sunny,60
5,"Partly cloudy, with a low around 51.",FridayNight,Partly Cloudy,51
6,"Mostly sunny, with a high near 61.",Saturday,Mostly Sunny,61
7,"Partly cloudy, with a low around 51.",SaturdayNight,Partly Cloudy,51
8,"Mostly sunny, with a high near 61.",Sunday,Mostly Sunny,61
